In [3]:
print('hello')


hello


In [1]:
%%time
# upgrade pip, make sure you are in the correct directory, and make sure you have the right python
#!python -m pip install --upgrade pip

import os
print(os.getcwd())
os.listdir()
#os.chdir('C:\Users\Christopher\Desktop\DataWarehouseProject\Design\Dimension_Color')
#os.chdir('C:\Users\Christopher\Desktop\Data Warehouse Project\Design\Dimension_Color')
print(os.getcwd())
# Check your version of python to make sure you are running python 3
import platform
print('Pyhton Version: ' + platform.python_version())

d:\django_dashboard_corona\secondDashboard\fact_sales_order_item
d:\django_dashboard_corona\secondDashboard\fact_sales_order_item
Pyhton Version: 3.8.10
Wall time: 998 µs


In [3]:
%%time
# importing libraries
import psycopg2
import pandas as pd
from pandas import DataFrame

# importing data from the data warehouse using psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=Darkknight17")
cur = conn.cursor()


cur.execute(""" 

--This is a query that creates a tempeorary table of the Detail line items data, enriched with  product data and color data
--Author: Christopher Gonzalez
--SELECT * FROM jcdw.dimorderdetailline
--SELECT * FROM 	

--SELECT * FROM jcdw.dimorderdetailline
--where orderitemid = '287463-12'

--SELECT * FROM jcdw.dimorderdetailline --25332 rows

----building temptable1--------------------------------------------------
DROP TABLE IF EXISTS  jcdw.tempTable1;

SELECT * FROM jcdw.dimorderdetailline;
SELECT * FROM jcdw.dimproduct;

SELECT 
  orderitemID, CustomerID, customername, salesOrder, lineItem, Sidemark,  Status
, Product,dimproduct.description AS productDescription, Model, WID
, HGT, Ordered AS quantityOrdered, Entered, Required , CreditOK
, Printed, Labels, Packed
, ShippedDate, Canceled
, ClrStyle, ShippedQty AS quantityShipped
, NetSale, customerPO, terms 
INTO TABLE jcdw.tempTable1
FROM jcdw.dimorderdetailline
JOIN jcdw.dimproduct
ON dimorderdetailline.product = dimproduct.productid;

--creating a temp color table to avoid duplicates -----------------------
DROP TABLE IF EXISTS  jcdw.tempColor;

SELECT DISTINCT ON (color)
        color, colordescription
INTO TABLE jcdw.tempColor
FROM   jcdw.dimcolor
ORDER  BY color;

----building temptable2-------------------------------------------------- 22688
DROP TABLE IF EXISTS  jcdw.temptable2 ;

SELECT * FROM jcdw.tempTable1;
SELECT * FROM jcdw.tempColor;

SELECT 
  orderitemID, CustomerID, customername, Sidemark, salesOrder, lineItem,  Status
, Product, productDescription, Model, WID
, HGT, quantityOrdered, Entered, Required , CreditOK
, Printed, Labels, Packed
, ShippedDate, Canceled
, ClrStyle,tempColor.colordescription AS colorDescription, quantityShipped
, NetSale, customerPO, terms 
INTO TABLE jcdw.temptable2
FROM jcdw.tempTable1
LEFT JOIN jcdw.tempColor
ON tempTable1.ClrStyle = tempColor.color;

----building ---------------------------------------------------------------------

SELECT * FROM jcdw.temptable2;
SELECT * FROM jcdw.FactOutSourcedItems;

insert INTO jcdw.FactOutSourcedItems (

            OutSourcedItemsID  , Status  
, Product , productDescription , terms 
, salesOrder , lineItem  , customerPO  
, CustomerID , customername , Sidemark 
, Entered , CreditOK  , Printed 
, Labels , Packed , ShippedDate 
, Required , Canceled , Model 
, ClrStyle , colorDescription, WID 
, HGT, quantityOrdered , quantityShipped 
, NetSale, trackingStatus, trackingNumber, Notes

)

SELECT 

  orderitemID AS OutSourcedItemsID, status
, product, productDescription, terms
, salesOrder, lineItem, customerPo
, CustomerID, customername, Sidemark
, entered, creditOk, printed
, labels, packed, shippedDate
, Required, canceled, model
, clrstyle, colorDescription, wid
, hgt, quantityOrdered, quantityShipped
, netSale, 'blank' AS trackingStatus, 'blank' AS trackingNumber, 'blank' AS notes

from jcdw.temptable2

ON CONFLICT ON CONSTRAINT Unique_OutSourcedItemsID DO UPDATE set 

  OutSourcedItemsID = excluded.OutSourcedItemsID, status = excluded.status
, product = excluded.product, productDescription = excluded.productDescription, terms = excluded.terms
, salesOrder = excluded.salesOrder, lineItem = excluded.lineItem, customerPo = excluded.customerPo
, CustomerID = excluded.CustomerID, customername = excluded.customername, Sidemark = excluded.Sidemark
, entered = excluded.entered, creditOk = excluded.creditOk, printed = excluded.printed
, labels = excluded.labels, packed = excluded.packed, shippedDate = excluded.shippedDate
, Required = excluded.Required, canceled = excluded.canceled, model = excluded.model
, clrstyle = excluded.clrstyle, colorDescription = excluded.colorDescription, wid = excluded.wid
, hgt = excluded.hgt, quantityOrdered = excluded.quantityOrdered, quantityShipped = excluded.quantityShipped
, netSale = excluded.netSale, trackingStatus = excluded.trackingStatus, trackingNumber = excluded.trackingNumber, notes = excluded.notes






""")


conn.commit()
conn.close()

# importing libraries
import psycopg2
import pandas as pd
from pandas import DataFrame

# importing data from the data warehouse using psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=Darkknight17")
cur = conn.cursor()
cur.execute("""SELECT * FROM jcdw.FactOutSourcedItems """)
rows = cur.fetchall()

print ("\nShow me the databases:\n")
for row in rows:
    print ("   ", row)

df = DataFrame(rows,columns=[
    
  'outsourceditemskey', 'outsourceditemsid', 'Status', 'Product', 'productDescription', 'terms', 'salesOrder',  'lineItem'
, 'customerPO', 'CustomerID', 'customername', 'Sidemark'
, 'Entered', 'CreditOK', 'Printed', 'Labels' , 'Packed'
, 'ShippedDate', 'Required', 'Canceled', 'Model', 'ClrStyle', 'colorDescription', 'WID', 'HGT', 'quantityOrdered', 'quantityShipped'
, 'NetSale', 'trackingStatus', 'trackingNumber', 'notes', 'updated'
    
     
    
])


conn.commit()
conn.close()
df

In [1]:
%%time
# importing libraries
import psycopg2
import pandas as pd
from pandas import DataFrame


# DATABASES = {
#     'default': {
#         'ENGINE': 'django.db.backends.postgresql_psycopg2',
#         'NAME': 'postgres',
#         'USER':'Christopher@econometricdatasolutionsdb',
#         'PASSWORD':'Darkknight17!',
#         'HOST':'econometricdatasolutionsdb.postgres.database.azure.com',
#         'PORT':'5432',
#         'OPTIONS': {'sslmode': 'require'},
#     }
# }

# importing data from the data warehouse using psycopg2
conn = psycopg2.connect("host='econometricdatasolutionsdb.postgres.database.azure.com' dbname=postgres user='Christopher@econometricdatasolutionsdb' password='Darkknight17!'")






# cur = conn.cursor()


# cur.execute(""" 

# --This is a query that creates a tempeorary table of the Detail line items data, enriched with  product data and color data
# --Author: Christopher Gonzalez
# --SELECT * FROM jcdw.dimorderdetailline
# --SELECT * FROM 	

# --SELECT * FROM jcdw.dimorderdetailline
# --where orderitemid = '287463-12'

# --SELECT * FROM jcdw.dimorderdetailline --25332 rows

# ----building temptable1--------------------------------------------------
# DROP TABLE IF EXISTS  jcdw.tempTable1;

# SELECT * FROM jcdw.dimorderdetailline;
# SELECT * FROM jcdw.dimproduct;

# SELECT 
#   orderitemID, CustomerID, customername, salesOrder, lineItem, Sidemark,  Status
# , Product,dimproduct.description AS productDescription, Model, WID
# , HGT, Ordered AS quantityOrdered, Entered, Required , CreditOK
# , Printed, Labels, Packed
# , ShippedDate, Canceled
# , ClrStyle, ShippedQty AS quantityShipped
# , NetSale, customerPO, terms 
# INTO TABLE jcdw.tempTable1
# FROM jcdw.dimorderdetailline
# JOIN jcdw.dimproduct
# ON dimorderdetailline.product = dimproduct.productid;

# --creating a temp color table to avoid duplicates -----------------------
# DROP TABLE IF EXISTS  jcdw.tempColor;

# SELECT DISTINCT ON (color)
#         color, colordescription
# INTO TABLE jcdw.tempColor
# FROM   jcdw.dimcolor
# ORDER  BY color;

# ----building temptable2-------------------------------------------------- 22688
# DROP TABLE IF EXISTS  jcdw.temptable2 ;

# SELECT * FROM jcdw.tempTable1;
# SELECT * FROM jcdw.tempColor;

# SELECT 
#   orderitemID, CustomerID, customername, Sidemark, salesOrder, lineItem,  Status
# , Product, productDescription, Model, WID
# , HGT, quantityOrdered, Entered, Required , CreditOK
# , Printed, Labels, Packed
# , ShippedDate, Canceled
# , ClrStyle,tempColor.colordescription AS colorDescription, quantityShipped
# , NetSale, customerPO, terms 
# INTO TABLE jcdw.temptable2
# FROM jcdw.tempTable1
# LEFT JOIN jcdw.tempColor
# ON tempTable1.ClrStyle = tempColor.color;

# ----building ---------------------------------------------------------------------

# SELECT * FROM jcdw.temptable2;
# SELECT * FROM jcdw.FactOutSourcedItems;

# insert INTO jcdw.FactOutSourcedItems (

#             OutSourcedItemsID  , Status  
# , Product , productDescription , terms 
# , salesOrder , lineItem  , customerPO  
# , CustomerID , customername , Sidemark 
# , Entered , CreditOK  , Printed 
# , Labels , Packed , ShippedDate 
# , Required , Canceled , Model 
# , ClrStyle , colorDescription, WID 
# , HGT, quantityOrdered , quantityShipped 
# , NetSale, trackingStatus, trackingNumber, Notes

# )

# SELECT 

#   orderitemID AS OutSourcedItemsID, status
# , product, productDescription, terms
# , salesOrder, lineItem, customerPo
# , CustomerID, customername, Sidemark
# , entered, creditOk, printed
# , labels, packed, shippedDate
# , Required, canceled, model
# , clrstyle, colorDescription, wid
# , hgt, quantityOrdered, quantityShipped
# , netSale, 'blank' AS trackingStatus, 'blank' AS trackingNumber, 'blank' AS notes

# from jcdw.temptable2

# ON CONFLICT ON CONSTRAINT Unique_OutSourcedItemsID DO UPDATE set 

#   OutSourcedItemsID = excluded.OutSourcedItemsID, status = excluded.status
# , product = excluded.product, productDescription = excluded.productDescription, terms = excluded.terms
# , salesOrder = excluded.salesOrder, lineItem = excluded.lineItem, customerPo = excluded.customerPo
# , CustomerID = excluded.CustomerID, customername = excluded.customername, Sidemark = excluded.Sidemark
# , entered = excluded.entered, creditOk = excluded.creditOk, printed = excluded.printed
# , labels = excluded.labels, packed = excluded.packed, shippedDate = excluded.shippedDate
# , Required = excluded.Required, canceled = excluded.canceled, model = excluded.model
# , clrstyle = excluded.clrstyle, colorDescription = excluded.colorDescription, wid = excluded.wid
# , hgt = excluded.hgt, quantityOrdered = excluded.quantityOrdered, quantityShipped = excluded.quantityShipped
# , netSale = excluded.netSale, trackingStatus = excluded.trackingStatus, trackingNumber = excluded.trackingNumber, notes = excluded.notes






# """)


# # conn.commit()
# conn.close()

# # importing libraries
# import psycopg2
# import pandas as pd
# from pandas import DataFrame

# # importing data from the data warehouse using psycopg2
# conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=Darkknight17")
# cur = conn.cursor()
# cur.execute("""SELECT * FROM jcdw.FactOutSourcedItems """)
# rows = cur.fetchall()

# print ("\nShow me the databases:\n")
# for row in rows:
#     print ("   ", row)

# df = DataFrame(rows,columns=[
    
#   'outsourceditemskey', 'outsourceditemsid', 'Status', 'Product', 'productDescription', 'terms', 'salesOrder',  'lineItem'
# , 'customerPO', 'CustomerID', 'customername', 'Sidemark'
# , 'Entered', 'CreditOK', 'Printed', 'Labels' , 'Packed'
# , 'ShippedDate', 'Required', 'Canceled', 'Model', 'ClrStyle', 'colorDescription', 'WID', 'HGT', 'quantityOrdered', 'quantityShipped'
# , 'NetSale', 'trackingStatus', 'trackingNumber', 'notes', 'updated'
    
     
    
# ])


# conn.commit()
# conn.close()
# df

Wall time: 1e+03 ms


In [3]:
%%time
# importing libraries
import psycopg2
import pandas as pd
from pandas import DataFrame

# importing data from the data warehouse using psycopg2
conn = psycopg2.connect("host='econometricdatasolutionsdb.postgres.database.azure.com' dbname=postgres user='Christopher@econometricdatasolutionsdb' password='Darkknight17!'")
cur = conn.cursor()
cur.execute("""SELECT * FROM jcwf_fact_sales_order_item """)
rows = cur.fetchall()

print ("\nShow me the databases:\n")
for row in rows:
    print ("   ", row)

df = DataFrame(rows) # ,columns=[
    
#   'outsourceditemskey', 'outsourceditemsid', 'Status', 'Product', 'productDescription', 'terms', 'salesOrder',  'lineItem'
# , 'customerPO', 'CustomerID', 'customername', 'Sidemark'
# , 'Entered', 'CreditOK', 'Printed', 'Labels' , 'Packed'
# , 'ShippedDate', 'Required', 'Canceled', 'Model', 'ClrStyle', 'colorDescription', 'WID', 'HGT', 'quantityOrdered', 'quantityShipped'
# , 'NetSale', 'trackingStatus', 'trackingNumber', 'notes', 'updated'
    
# ])


conn.commit()
conn.close()
df



Show me the databases:

    (1, 'Fabricated & Components', '1 JC WINDOW FASHIONS', '1 JC WINDOW FASHIONS', '30 BATCHED', '1', 'COD', '297568-2', None, '1', 'LEXUS', '2021-08-11 00:00:00', '2021-08-11 00:00:00', '2021-08-19 00:00:00', None, '2021-08-19 00:00:00', '2021-08-23 00:00:00', '2021-08-26 00:00:00', None, 'SOMFY', None, 'N/A', 'N/A', '1', '0', '58', None, 'Y', datetime.datetime(2021, 9, 19, 22, 24, 16), datetime.datetime(2021, 9, 19, 22, 24, 16), 'original state', 'v 1.0', '0', 'admin')
    (2, 'Fabricated & Components', '1 JC WINDOW FASHIONS', '1 JC WINDOW FASHIONS', '30 BATCHED', '1', None, '297701-10', None, '1', 'IRVINE', '2021-08-10 00:00:00', '2021-08-11 00:00:00', '2021-09-01 00:00:00', None, '2021-09-17 00:00:00', '2021-09-17 00:00:00', '2021-09-01 00:00:00', None, 'SOMFY', None, 'N/A', 'N/A', '1', '0', '53', None, 'Y', datetime.datetime(2021, 9, 19, 22, 24, 17), datetime.datetime(2021, 9, 19, 22, 24, 17), 'original state', 'v 1.0', '0', 'admin')
    (3, 'Fabricated & 

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,1,Fabricated & Components,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,30 BATCHED,1,COD,297568-2,None,1,...,0,58,None,Y,2021-09-19 22:24:16,2021-09-19 22:24:16,original state,v 1.0,0,admin
1,2,Fabricated & Components,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,30 BATCHED,1,None,297701-10,None,1,...,0,53,None,Y,2021-09-19 22:24:17,2021-09-19 22:24:17,original state,v 1.0,0,admin
2,3,Fabricated & Components,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,30 BATCHED,1,None,297701-11,None,1,...,0,133,None,Y,2021-09-19 22:24:17,2021-09-19 22:24:17,original state,v 1.0,0,admin
3,4,Fabricated & Components,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,30 BATCHED,1,COD,297935-20,None,1,...,0,53,None,Y,2021-09-19 22:24:17,2021-09-19 22:24:17,original state,v 1.0,0,admin
4,5,Fabricated & Components,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,30 BATCHED,1,COD,297935-21,None,1,...,0,15,None,Y,2021-09-19 22:24:17,2021-09-19 22:24:17,original state,v 1.0,0,admin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,311,Outsourced & Miscellaneous,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,71 FULL PACKED,1,None,298422-1,TOP-210833,1,...,0,180.68,None,Y,2021-09-19 22:24:32,2021-09-19 22:24:32,original state,v 1.0,0,admin
311,312,Outsourced & Miscellaneous,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,71 FULL PACKED,1,None,298945-1,None,1,...,0,0,None,Y,2021-09-19 22:24:33,2021-09-19 22:24:33,original state,v 1.0,0,admin
312,313,Outsourced & Miscellaneous,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,81 FULLY SHIPPED,1,COD,298454-5,None,1,...,1,83.37,None,Y,2021-09-19 22:24:33,2021-09-19 22:24:33,original state,v 1.0,0,admin
313,314,Outsourced & Miscellaneous,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,81 FULLY SHIPPED,1,COD,298454-6,None,1,...,1,83.37,None,Y,2021-09-19 22:24:33,2021-09-19 22:24:33,original state,v 1.0,0,admin


In [7]:
%%time
# importing libraries
import psycopg2
import pandas as pd
from pandas import DataFrame

# importing data from the data warehouse using psycopg2
conn = psycopg2.connect("host='econometricdatasolutionsdb.postgres.database.azure.com' dbname=postgres user='Christopher@econometricdatasolutionsdb' password='Darkknight17!'")
cur = conn.cursor()
cur.execute("""
  
/* Updating transformation table */
insert INTO jcwf_tran_sales_order_item (

  sales_order_item_product_category  
, sales_order_item_warehouse  
, sales_order_item_site 
, sales_order_item_status 
, sales_order_item_product 
, sales_order_item_product_foreign_key
, sales_order_item_terms 
, sales_order_item_order_item  
, sales_order_item_po  
, sales_order_item_customer_id 
, sales_order_item_customer_name 
, sales_order_item_sidemark 
, sales_order_item_entered 
, sales_order_item_credit_ok  
, sales_order_item_printed 
, sales_order_item_labels 
, sales_order_item_packed 
, sales_order_item_shipped_date 
, sales_order_item_required 
, sales_order_item_canceled 
, sales_order_item_model 
, sales_order_item_color_style 
, sales_order_item_width 
, sales_order_item_height
, sales_order_item_ordered 
, sales_order_item_shipped_quantity 
, sales_order_item_net_sale
, sales_order_item_cost_of_good_sold


)

SELECT 

  sales_order_item_product_category  
, sales_order_item_warehouse  
, sales_order_item_site 
, sales_order_item_status 
, sales_order_item_product 
, (SELECT product_key FROM jcwf_dim_product WHERE jcwf_dim_product.product_id = jcwf_stage_sales_order_item.sales_order_item_product) AS sales_order_item_product_foreign_key
, sales_order_item_terms 
, sales_order_item_order_item  
, sales_order_item_po  
, sales_order_item_customer_id 
, sales_order_item_customer_name 
, sales_order_item_sidemark 
, sales_order_item_entered 
, sales_order_item_credit_ok  
, sales_order_item_printed 
, sales_order_item_labels 
, sales_order_item_packed 
, sales_order_item_shipped_date 
, sales_order_item_required 
, sales_order_item_canceled 
, sales_order_item_model 
, sales_order_item_color_style 
, sales_order_item_width 
, sales_order_item_height
, sales_order_item_ordered 
, sales_order_item_shipped_quantity 
, sales_order_item_net_sale
, sales_order_item_cost_of_good_sold

from jcwf_stage_sales_order_item

ON CONFLICT ON CONSTRAINT Unique_jcwf_tran_sales_order_item_id DO UPDATE set 

  sales_order_item_product_category = excluded.sales_order_item_product_category
, sales_order_item_warehouse = excluded.sales_order_item_warehouse
, sales_order_item_site = excluded.sales_order_item_site
, sales_order_item_status = excluded.sales_order_item_status
, sales_order_item_product = excluded.sales_order_item_product
, sales_order_item_product_foreign_key = excluded.sales_order_item_product_foreign_key
, sales_order_item_terms = excluded.sales_order_item_terms
, sales_order_item_order_item = excluded.sales_order_item_order_item
, sales_order_item_po = excluded.sales_order_item_po
, sales_order_item_customer_id = excluded.sales_order_item_customer_id
, sales_order_item_customer_name = excluded.sales_order_item_customer_name
, sales_order_item_sidemark = excluded.sales_order_item_sidemark
, sales_order_item_entered = excluded.sales_order_item_entered
, sales_order_item_credit_ok = excluded.sales_order_item_credit_ok
, sales_order_item_printed = excluded.sales_order_item_printed
, sales_order_item_labels = excluded.sales_order_item_labels
, sales_order_item_packed = excluded.sales_order_item_packed
, sales_order_item_shipped_date = excluded.sales_order_item_shipped_date
, sales_order_item_required = excluded.sales_order_item_required
, sales_order_item_canceled = excluded.sales_order_item_canceled
, sales_order_item_model = excluded.sales_order_item_model
, sales_order_item_color_style = excluded.sales_order_item_color_style
, sales_order_item_width = excluded.sales_order_item_width
, sales_order_item_height = excluded.sales_order_item_height
, sales_order_item_ordered = excluded.sales_order_item_ordered
, sales_order_item_shipped_quantity = excluded.sales_order_item_shipped_quantity
, sales_order_item_net_sale = excluded.sales_order_item_net_sale
, sales_order_item_cost_of_good_sold = excluded.sales_order_item_cost_of_good_sold;

-------------------------------------------------------------------------------------------------------
/* Updating fact table */
insert INTO jcwf_fact_sales_order_item (

  sales_order_item_product_category  
, sales_order_item_warehouse  
, sales_order_item_site 
, sales_order_item_status 
, sales_order_item_product_foreign_key
, sales_order_item_terms 
, sales_order_item_order_item  
, sales_order_item_po  
, sales_order_item_customer_id_foreign_key 
, sales_order_item_sidemark 
, sales_order_item_entered 
, sales_order_item_credit_ok  
, sales_order_item_printed 
, sales_order_item_labels 
, sales_order_item_packed 
, sales_order_item_shipped_date 
, sales_order_item_required 
, sales_order_item_canceled 
, sales_order_item_model 
, sales_order_item_color_foreign_key 
, sales_order_item_width 
, sales_order_item_height
, sales_order_item_ordered 
, sales_order_item_shipped_quantity 
, sales_order_item_net_sale
, sales_order_item_cost_of_good_sold


)

SELECT 

  sales_order_item_product_category  
, sales_order_item_warehouse  
, sales_order_item_site 
, sales_order_item_status 
, sales_order_item_product_foreign_key
, sales_order_item_terms 
, sales_order_item_order_item  
, sales_order_item_po  
, sales_order_item_customer_id_foreign_key 
, sales_order_item_sidemark 
, sales_order_item_entered 
, sales_order_item_credit_ok  
, sales_order_item_printed 
, sales_order_item_labels 
, sales_order_item_packed 
, sales_order_item_shipped_date 
, sales_order_item_required 
, sales_order_item_canceled 
, sales_order_item_model 
, sales_order_item_color_foreign_key 
, sales_order_item_width 
, sales_order_item_height
, sales_order_item_ordered 
, sales_order_item_shipped_quantity 
, sales_order_item_net_sale
, sales_order_item_cost_of_good_sold

from jcwf_tran_sales_order_item

ON CONFLICT ON CONSTRAINT Unique_fact_sales_order_item DO UPDATE set 

  sales_order_item_product_category = excluded.sales_order_item_product_category
, sales_order_item_warehouse = excluded.sales_order_item_warehouse
, sales_order_item_site = excluded.sales_order_item_site
, sales_order_item_status = excluded.sales_order_item_status
, sales_order_item_product_foreign_key = excluded.sales_order_item_product_foreign_key
, sales_order_item_terms = excluded.sales_order_item_terms
, sales_order_item_order_item = excluded.sales_order_item_order_item
, sales_order_item_po = excluded.sales_order_item_po
, sales_order_item_customer_id_foreign_key = excluded.sales_order_item_customer_id_foreign_key
, sales_order_item_sidemark = excluded.sales_order_item_sidemark
, sales_order_item_entered = excluded.sales_order_item_entered
, sales_order_item_credit_ok = excluded.sales_order_item_credit_ok
, sales_order_item_printed = excluded.sales_order_item_printed
, sales_order_item_labels = excluded.sales_order_item_labels
, sales_order_item_packed = excluded.sales_order_item_packed
, sales_order_item_shipped_date = excluded.sales_order_item_shipped_date
, sales_order_item_required = excluded.sales_order_item_required
, sales_order_item_canceled = excluded.sales_order_item_canceled
, sales_order_item_model = excluded.sales_order_item_model
, sales_order_item_color_foreign_key = excluded.sales_order_item_color_foreign_key
, sales_order_item_width = excluded.sales_order_item_width
, sales_order_item_height = excluded.sales_order_item_height
, sales_order_item_ordered = excluded.sales_order_item_ordered
, sales_order_item_shipped_quantity = excluded.sales_order_item_shipped_quantity
, sales_order_item_net_sale = excluded.sales_order_item_net_sale
, sales_order_item_cost_of_good_sold = excluded.sales_order_item_cost_of_good_sold;

SELECT * FROM jcwf_fact_sales_order_item;

"""





)



rows = cur.fetchall()

print ("\nShow me the databases:\n")
for row in rows:
    print ("   ", row)

df = DataFrame(rows) # ,columns=[
    
#   'outsourceditemskey', 'outsourceditemsid', 'Status', 'Product', 'productDescription', 'terms', 'salesOrder',  'lineItem'
# , 'customerPO', 'CustomerID', 'customername', 'Sidemark'
# , 'Entered', 'CreditOK', 'Printed', 'Labels' , 'Packed'
# , 'ShippedDate', 'Required', 'Canceled', 'Model', 'ClrStyle', 'colorDescription', 'WID', 'HGT', 'quantityOrdered', 'quantityShipped'
# , 'NetSale', 'trackingStatus', 'trackingNumber', 'notes', 'updated'
    
# ])


conn.commit()
conn.close()
df


Show me the databases:

    (15, 'Fabricated & Components', '1 JC WINDOW FASHIONS', '1 JC WINDOW FASHIONS', '30 BATCHED', 45, 'COD', '297935-12', None, 1, 'MR TIM', '2021-08-12 00:00:00', '2021-08-17 00:00:00', '2021-09-13 00:00:00', None, '2021-09-14 00:00:00', '2021-09-14 00:00:00', '2021-09-22 00:00:00', None, 'CLUTCH', None, '34.625', '57.25', '1', '0', '123.42', None, 'Y', datetime.datetime(2021, 9, 19, 23, 13, 6), datetime.datetime(2021, 9, 19, 23, 13, 6), 'original state', 'v 1.0', '0', 'admin')
    (16, 'Fabricated & Components', '1 JC WINDOW FASHIONS', '1 JC WINDOW FASHIONS', '30 BATCHED', 45, 'COD', '297935-13', None, 1, 'MR TIM', '2021-08-12 00:00:00', '2021-08-17 00:00:00', '2021-09-13 00:00:00', None, '2021-09-14 00:00:00', '2021-09-14 00:00:00', '2021-09-22 00:00:00', None, 'CLUTCH', None, '33.125', '45', '1', '0', '119.01', None, 'Y', datetime.datetime(2021, 9, 19, 23, 13, 6), datetime.datetime(2021, 9, 19, 23, 13, 6), 'original state', 'v 1.0', '0', 'admin')
    (17, '

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,15,Fabricated & Components,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,30 BATCHED,45.0,COD,297935-12,None,1,...,0,123.42,None,Y,2021-09-19 23:13:06,2021-09-19 23:13:06,original state,v 1.0,0,admin
1,16,Fabricated & Components,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,30 BATCHED,45.0,COD,297935-13,None,1,...,0,119.01,None,Y,2021-09-19 23:13:06,2021-09-19 23:13:06,original state,v 1.0,0,admin
2,17,Fabricated & Components,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,30 BATCHED,45.0,COD,297935-15,None,1,...,0,168.53,None,Y,2021-09-19 23:13:06,2021-09-19 23:13:06,original state,v 1.0,0,admin
3,1,Fabricated & Components,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,30 BATCHED,5.0,COD,297568-2,None,1,...,0,58,None,Y,2021-09-19 23:13:05,2021-09-19 23:13:05,original state,v 1.0,0,admin
4,2,Fabricated & Components,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,30 BATCHED,5.0,None,297701-10,None,1,...,0,53,None,Y,2021-09-19 23:13:05,2021-09-19 23:13:05,original state,v 1.0,0,admin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,304,Outsourced & Miscellaneous,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,71 FULL PACKED,90.0,None,298299-32,None,1,...,0,98.54,None,Y,2021-09-19 23:13:20,2021-09-19 23:13:20,original state,v 1.0,0,admin
311,305,Outsourced & Miscellaneous,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,71 FULL PACKED,90.0,None,298299-33,None,1,...,0,139.16,None,Y,2021-09-19 23:13:20,2021-09-19 23:13:20,original state,v 1.0,0,admin
312,306,Outsourced & Miscellaneous,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,71 FULL PACKED,90.0,None,298299-34,None,1,...,0,102.43,None,Y,2021-09-19 23:13:20,2021-09-19 23:13:20,original state,v 1.0,0,admin
313,307,Outsourced & Miscellaneous,1 JC WINDOW FASHIONS,1 JC WINDOW FASHIONS,71 FULL PACKED,90.0,None,298299-35,None,1,...,0,111.96,None,Y,2021-09-19 23:13:20,2021-09-19 23:13:20,original state,v 1.0,0,admin
